In [1]:
import random
import matplotlib.pyplot as plt
import math
import numpy as np
import heapq
from scipy.stats import t
import scipy.stats

In [2]:
class Station:
    def __init__(self, size):
        self.size=size
        self.spaces=[0]*size
        self.happy_ships=0
        self.sad_ships=0
    
    def dock_ship(self):
        try:
            empty_space = self.spaces.index(0)
            self.spaces[empty_space] = 1
            self.happy_ships+=1
            return True
        except ValueError:
            self.sad_ships+=1
            return False
        
    def depart_ship(self):
        try:
            full_space = self.spaces.index(1)
            self.spaces[full_space]=0
            return True
        except ValueError:
            return False
    
    def print_details(self):
        print('{} happy ships'.format(self.happy_ships))
        print('{} sad ships'.format(self.sad_ships))
        print('{} total ships'.format(self.sad_ships+self.happy_ships))
        
    
    def b_formula(self):
        return(self.sad_ships/((self.sad_ships+self.happy_ships)))

In [3]:
def generate_hyperexponential(p1, l1, p2, l2):
    while True:
        u = random.random()
        if u < p1:
            return random.expovariate(l1)
        else:
            return random.expovariate(l2)

In [21]:
def run_simulation(mode):
    
    RUNS = 10000
    
    MEAN_TIME_BETWEEN_CUSTOMERS = 1
    MEAN_SERVICE_TIME = 8

    p1 = 0.8
    l1 = 0.8333
    
    p2 = 0.2
    l2 = 5.0
    
    K1 = 1.05
    K2 = 2.05

    # mode=1 -> Default :
    # - The arrival process is modelled as a Poisson process 
    # - The service time distribution is chosen as exponential.

    # mode=2.1 -> Erlang distributed inter arrival times with a mean of 1
    # mode=2.2 -> Hyper exponential inter arrival times

    
    jess_station = Station(10)
    arrival_times = []
    

    if(mode==2.1):
        arrival_times = scipy.stats.erlang.rvs(1, size = RUNS)
    elif(mode==2.2):
        arrival_times = [generate_hyperexponential(p1, l1, p2, l2) for _ in range(RUNS)]
    else:
        ## Default
        arrival_times = np.random.exponential(MEAN_TIME_BETWEEN_CUSTOMERS, size=10000)

    
    arrival_schedule = np.cumsum(arrival_times)
    
    departure_schedule = []
    num_arrivals = len(arrival_schedule)
    num_departures = 0
    num_ships_docked = 0

    for i in range(round(RUNS*1.1)):        
        arriving_ships = np.argwhere(np.floor(arrival_schedule) == i).flatten()
        num_arriving_ships = len(arriving_ships)
        
        for ship in range(num_arriving_ships):
            if jess_station.dock_ship():
                
                rand_departure_time = 0                
                if(mode==3.1):
                    # Constant service time
                    rand_departure_time = 10
                elif(mode==3.2):
                    # Pareto K=1.05
                    rand_departure_time = np.random.pareto(K1,1)+1
                elif(mode==3.3):
                    # Pareto K=2.05
                    rand_departure_time = np.random.pareto(K2,1)+1
                elif(mode==3.4):
                    # Pareto K=2.05
                    dist = [1,2,9,10]
                    rand_departure_time = np.random.choice(dist)
                else:
                    ## Default
                    rand_departure_time = np.random.exponential(MEAN_SERVICE_TIME)
                
                heapq.heappush(departure_schedule, i + rand_departure_time)
                num_ships_docked += 1
        while departure_schedule and np.floor(departure_schedule[0]) == i:
            heapq.heappop(departure_schedule)
            jess_station.depart_ship()
            num_departures += 1
            
    return jess_station.b_formula()

In [18]:
CONFIDENCE_LEVEL = 0.95

exact_solution = (pow(8,10)/math.factorial(10)) / np.sum([pow(8,i)/math.factorial(i) for i in range(0,11)])                                                         
print('Theoretical Solution B-Value:', exact_solution)

def calculate_confidence_interval(results):   
    mean = np.mean(results)
    std = np.std(results)
    degrees_of_freedom = len(results) - 1
    critical_value = t.ppf((1 + CONFIDENCE_LEVEL) / 2, degrees_of_freedom)
    margin_of_error = critical_value * (std / np.sqrt(len(results)))
    confidence_interval = (mean - margin_of_error, mean + margin_of_error)
    
    print("Mean B-Value:", mean)
    print("Standard Deviation:", std)
    print("Confidence Interval:", confidence_interval)

Theoretical Solution B-Value: 0.12166106425295149


In [25]:
def print_header(question):
    separator = "#" * 11
    print(separator)
    print(question)
    print(separator)

print_header("Question 1")
calculate_confidence_interval([run_simulation(1) for _ in range(10)])
print()  # Empty line

print_header("Question 2a")
calculate_confidence_interval([run_simulation(2.1) for _ in range(10)])
print()

print_header("Question 2b")
calculate_confidence_interval([run_simulation(2.2) for _ in range(10)])
print()

print_header("Question 3a")
calculate_confidence_interval([run_simulation(3.1) for _ in range(10)])
print()

print_header("Question 3b")
print("With K=1.05")
calculate_confidence_interval([run_simulation(3.2) for _ in range(10)])
print("With K=2.05")
calculate_confidence_interval([run_simulation(3.3) for _ in range(10)])
print()

print_header("Question 3c")
calculate_confidence_interval([run_simulation(3.4) for _ in range(10)])


###########
Question 1
###########
Mean B-Value: 0.12721999999999997
Standard Deviation: 0.005924322746103553
Confidence Interval: (0.12298199481048289, 0.13145800518951706)

###########
Question 2a
###########
Mean B-Value: 0.12523
Standard Deviation: 0.008357876524572497
Confidence Interval: (0.11925113530904818, 0.13120886469095183)

###########
Question 2b
###########
Mean B-Value: 0.14032999999999998
Standard Deviation: 0.005055501953317795
Confidence Interval: (0.1367135117646374, 0.14394648823536257)

###########
Question 3a
###########
Mean B-Value: 0.24013999999999996
Standard Deviation: 0.004199095140622559
Confidence Interval: (0.23713614829240373, 0.2431438517075962)

###########
Question 3b
###########
With K=1.05
Mean B-Value: 0.15960000000000002
Standard Deviation: 0.04836701355262695
Confidence Interval: (0.12500032283482668, 0.19419967716517336)
With K=2.05
Mean B-Value: 5.9999999999999995e-05
Standard Deviation: 0.0001019803902718557
Confidence Interval: (-1.295237645